In [53]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

In [54]:
installs = pd.read_csv('datos/installs.csv.gzip', low_memory=False, compression='gzip', parse_dates = ['created'], index_col=['created'])
installs = installs.sort_index()
tiempo_max = 60*60*24*3

In [55]:
test_y_temp = installs['2019-04-20':'2019-04-22']
test_y_temp = test_y_temp.reset_index()
test_y_temp['created'] = test_y_temp['created'] - pd.Timestamp(f'2019-04-20 00:00:00')
test_y_temp['created'] = test_y_temp['created'].transform(lambda x: x.total_seconds())
test_y_temp = test_y_temp.groupby('ref_hash').agg({'created' : 'min'}).reset_index()
test_y_temp = test_y_temp[["ref_hash", "created"]]

In [56]:
test_y = pd.read_csv("final.csv", names=["ref_hash"])

In [57]:
hashes = test_y_temp["ref_hash"]
hashes_norm = (hashes-hashes.mean())/hashes.std()
train_x_scaled = hashes_norm
train_y = test_y_temp["created"]

In [58]:
reg = MLPRegressor(verbose=True, random_state=123,
                                          activation="relu",
                                          solver="adam",
                                          alpha=0.01,
                                        max_iter=70)

In [59]:
train_reshaped = np.array(train_x_scaled).reshape(-1,1)

In [60]:
reg.fit(train_reshaped, train_y)

Iteration 1, loss = 11576366108.18711472
Iteration 2, loss = 11559936411.48910904
Iteration 3, loss = 11529122923.94842529
Iteration 4, loss = 11486830366.32990074
Iteration 5, loss = 11434767506.96130753
Iteration 6, loss = 11374017605.53965759
Iteration 7, loss = 11305275017.08534050
Iteration 8, loss = 11229016381.96932793
Iteration 9, loss = 11145670591.30950546
Iteration 10, loss = 11055516753.05479431
Iteration 11, loss = 10958773361.83776283
Iteration 12, loss = 10855641891.75649643
Iteration 13, loss = 10746372616.28464317
Iteration 14, loss = 10631120850.41031265
Iteration 15, loss = 10510121062.03639793
Iteration 16, loss = 10383547384.22431755
Iteration 17, loss = 10251697205.54758453
Iteration 18, loss = 10114702519.68172836
Iteration 19, loss = 9972807993.08614922
Iteration 20, loss = 9826253038.25864983
Iteration 21, loss = 9675176862.92447281
Iteration 22, loss = 9519945363.31062317
Iteration 23, loss = 9360740436.27254677
Iteration 24, loss = 9197786112.58825684
Iterati

/home/julian/.local/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (70) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPRegressor(activation='relu', alpha=0.01, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=70, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=123, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=True, warm_start=False)

In [68]:
target_norm = (test_y-test_y.mean())/test_y.std()
target_norm = np.array(target_norm).reshape(-1,1)

In [69]:
predicciones = reg.predict(target_norm)

In [71]:
max(predicciones)

146283.52378673185

In [72]:
auxiliar = test_y.copy()
auxiliar["prediccion"] = predicciones

In [73]:
resultados = auxiliar.merge(test_y, how="right").fillna(60*60*24*3)

In [74]:
resultados.columns = ["ref_hash", "obj"]

In [75]:
resultados.to_csv("prediccion_mlp_simple_otra_vent.csv", index=False)